In [1]:
# Notebook to read in to the results of AiZynthFinders and parse reaction trees to json for further analysis

In [2]:
import pandas as pd
import json
import sys

sys.path.append('../../')

In [3]:
#import aizynthfinder.chem as chem
import aizynthfinder.analysis

from aizynthfinder.mcts.node import Node
from aizynthfinder.analysis import ReactionTree
from aizynthfinder.mcts.state import State
from aizynthfinder.chem import TreeMolecule
from aizynthfinder.context.collection import ContextCollection
from aizynthfinder.context.stock import StockException

In [4]:
import pandas as pd

data = pd.read_hdf('/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/explore/results/tg1_std_1.hdf5', 'table')

In [5]:
print(data.head)

<bound method NDFrame.head of                                                target  search_time  \
0     O=S(=O)(NCC(c1ccco1)N1CCN(c2ccccc2)CC1)c1ccccc1    81.547154   
1        Cc1ccc(C(CNS(=O)(=O)c2ccccc2)N2CCN(C)CC2)cc1   116.712183   
2       O=S(=O)(NCC(c1ccccc1)N1CCCCCC1)c1ccc(F)c(F)c1   120.173156   
3   CCCS(=O)(=O)c1ccc(C(CNS(=O)(=O)c2ccc(C(F)(F)F)...   112.410574   
4    Cc1ccc(S(=O)(=O)NCC(c2ccccc2)N2CCN(CCC#N)CC2)cc1   116.261097   
..                                                ...          ...   
12  O=S(=O)(NCC(c1cccnc1)N1CCN(Cc2ccccc2)CC1)c1ccc...    70.618498   
13                c1ccc(C[C@@H](c2ccccc2)N2CCCCC2)cc1    40.119433   
14            O=S(=O)(NCC(c1ccccc1)N1CCCCCC1)c1cccnc1   117.678594   
15     O=S(=O)(NCC(c1ccccc1)N1CCCCCC1)c1ccc(F)c(Cl)c1   120.622877   
16  NS(=O)(=O)c1ccc(C(CNS(=O)(=O)c2ccc(C(F)(F)F)cc...   120.473612   

    number_of_nodes  max_transforms  max_children  number_of_leafs  \
0               735               7            46          

In [6]:
solved_data = data.loc[(data.is_solved==True)]
unsolved_data = data.loc[(data.is_solved==False)]
all_solved = data.is_solved.values

In [7]:
true = []
false = []

for i in range(len(all_solved)):
    if all_solved[i] == True:
        true.append(i)
    else:
        false.append(i)


In [8]:
print('True:', (len(true)/len(all_solved))*100)
print('False: ', (len(false)/len(all_solved))*100)

True: 98.87640449438202
False:  1.1235955056179776


In [9]:
from aizynthfinder.analysis import ReactionTree

all_trees = data.trees.values
all_solved_trees = solved_data.trees.values
all_unsolved_trees = unsolved_data.trees.values
first_molecule_tree = all_trees[0]

print(first_molecule_tree)



ren': [{'type': 'reaction', 'hide': False, 'smiles': '[c:1]([cH2:2])([cH2:3])[N:5]([CH3:4])[CH3:6]>>F[c:1]([cH2:2])[cH2:3].[CH3:4][N:5][CH3:6]', 'is_reaction': True, 'metadata': {'template_hash': 'd6ea9b633b6bf4e6067258d496eb35f7d14e90bca4edf5fe58c28c1b', 'classification': 'N-arylation with Ar-X', 'library_occurence': 1380, 'policy_probability': 0.010499999858438969, 'policy_name': 'uspto', 'template_code': 39330}, 'children': [{'type': 'mol', 'hide': False, 'smiles': 'Fc1ccccc1', 'is_chemical': True, 'in_stock': True}, {'type': 'mol', 'hide': False, 'smiles': 'O=S(=O)(NCC(c1ccco1)N1CCNCC1)c1ccccc1', 'is_chemical': True, 'in_stock': False, 'children': [{'type': 'reaction', 'hide': False, 'smiles': '[CH3:1][N:2][CH3:3]>>c1ccc(C[N:2]([CH3:1])[CH3:3])cc1', 'is_reaction': True, 'metadata': {'template_hash': 'f58207c6c64a5b48e40756ac3ba50fe69cdf9b1d18b6063ebd95ff15', 'classification': 'NH deprotections', 'library_occurence': 1854, 'policy_probability': 0.1388999968767166, 'policy_name': 'us

In [10]:
json_results = []
solved_json_results = []
unsolved_json_results = []

for i in all_trees:
    for itree, tree in enumerate(i):
        reaction_json = ReactionTree.from_dict(tree).to_json()
        json_results.append(reaction_json)

for i in all_unsolved_trees:
    for itree, tree in enumerate(i):
        unsolved_reaction_json = ReactionTree.from_dict(tree).to_json()
        unsolved_json_results.append(unsolved_reaction_json)

for i in all_solved_trees:
    for itree, tree in enumerate(i):
        solved_reaction_json = ReactionTree.from_dict(tree).to_json()
        solved_json_results.append(solved_reaction_json)



In [11]:
# save reaction trees to file

data = {
    'reaction trees': solved_json_results
}

with open('/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/explore/results/tg1_std_solved.json', 'w') as outfile:
    json.dump(data, outfile)

In [13]:
print(solved_json_results[2])

{
  "type": "mol",
  "hide": false,
  "smiles": "O=S(=O)(NCC(c1ccco1)N1CCN(c2ccccc2)CC1)c1ccccc1",
  "is_chemical": true,
  "in_stock": false,
  "children": [
    {
      "type": "reaction",
      "hide": false,
      "smiles": "[c:1]([cH2:2])[cH2:3]>>N#C[c:1]([cH2:2])[cH2:3]",
      "is_reaction": true,
      "metadata": {
        "template_hash": "741c4d3be97c223dbcadd0d058f632a979821dd0ad35e03f37d4a4a2",
        "classification": "Unassigned",
        "library_occurence": 14,
        "policy_probability": 0.0003000000142492354,
        "policy_name": "uspto",
        "template_code": 21326
      },
      "children": [
        {
          "type": "mol",
          "hide": false,
          "smiles": "N#Cc1ccccc1S(=O)(=O)NCC(c1ccco1)N1CCN(c2ccccc2)CC1",
          "is_chemical": true,
          "in_stock": false,
          "children": [
            {
              "type": "reaction",
              "hide": false,
              "smiles": "[S:1](=[O:2])(=[O:3])([cH3:4])[N:6][CH3:5]>>Cl[S:1]

In [14]:
print(len(solved_json_results))

545
